In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
# tf.compat.v1.enable_eager_execution()
# from tensorflow.python.ops.numpy_ops import np_config
# np_config.enable_numpy_behavior()
# tf.executing_eagerly()

In [4]:
tf.executing_eagerly()

True

In [5]:
#Maximum sequence length including padding
global MAXLEN
MAXLEN = 400

# Load data from csv

In [6]:
if "x1TrainRnnS" not in locals():
    train = pd.read_pickle("../data/proccesed_train_set.pkl")
    x1TrainRnnS, x2TrainRnnS = train[["x1", "x2"]].applymap(lambda x: np.asarray(x).astype('float32')).values.T
    yTrainRnnS = train["y"].astype('float32')
    test = pd.read_pickle("../data/proccesed_test_set.pkl")
    x1ValRnnS, x2ValRnnS = test[["x1", "x2"]].applymap(lambda x: np.asarray(x).astype('float32')).values.T
    yValRnnS = test["y"].astype('float32')
    del train
    del test

In [7]:
# del x1TrainRnnS

--------------------------------

In [8]:
# x, y = tf.constant([[[1.0,2.0,3.0], [1.0,2.0,3.0]],[[3.0,4.0,5.0],[3.0,4.0,5.0]]])

In [9]:
@tf.function(jit_compile=True)  # The decorator converts `cosine_similarity` into a tensolflow `Function`.
def cosine_similarity(vects: tf.TensorArray) -> tf.TensorArray:
    '''Cosine similarity to be calculated as sum(x*y)/(sqrt(sum(x))*sqrt(sum(y))).
    This is achieved through Tensorflow functions to retain performance.
    
    Parameters
    ----------
    vects: tf.TensorArray
        
    
    Returns
    -------
    cosine_distance: tf.TensorArray
       The result of the cosine similarity between the vectors.    
    '''
    x, y = vects
    return tf.math.divide(tf.reduce_sum(tf.multiply(x,y), axis=1, keepdims=True), tf.multiply(tf.norm(x, ord=2, axis=1, keepdims=True), tf.norm(y, ord=2, axis=1, keepdims=True)))

In [10]:
# tf.assert_less(cosine_similarity(tf.constant([[[1.0,2.0,3.0]],[[3.0,4.0,5.0]]]))[0][0], 0.982708)

In [11]:
@tf.function(jit_compile=True)  # The decorator converts `cosine_distance` into a tensolflow `Function`.
def cosine_distance(vects: tf.TensorArray) -> tf.TensorArray:
    '''Cosine distance to be calculated as 1-(cosine similarity).
    Where cosine similarity equals sum(x*y)/(sqrt(sum(x))*sqrt(sum(y))).
    This is achieved through Tensorflow functions to retain performance.
    
    Parameters
    ----------
    vects: tf.TensorArray
        
    
    Returns
    -------
    cosine_distance: tf.TensorArray
        The result of 1-cosine similarity between the vectors. 
    '''
    x, y = vects
    return tf.math.subtract(tf.constant([1.0]), tf.math.divide(tf.reduce_sum(tf.multiply(x,y), axis=1, keepdims=True), tf.multiply(tf.norm(x, ord=2, axis=1, keepdims=True), tf.norm(y, ord=2, axis=1, keepdims=True))))

In [12]:
# tf.assert_greater(cosine_distance(tf.constant([[[1.0,2.0,3.0]],[[3.0,4.0,5.0]]]))[0][0], 1 - 0.982708)

In [13]:
class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self, margin: tf.float32 = 1.0):
        super(ContrastiveLoss, self).__init__()
        self.name = 'ContrastiveLoss'
        self.margin = tf.constant(margin)

    @tf.function(jit_compile=True)  # The decorator converts `loss` into a tensolflow `Function`.
    def call(self, y_true: tf.TensorArray, y_pred: tf.TensorArray) -> tf.Tensor:
        return tf.math.reduce_mean((1 - y_true) * tf.math.square(y_pred) + (y_true) * tf.math.square(tf.math.maximum(self.margin - (y_pred), 0.0)), axis = -1)

In [14]:
# a = ContrastiveLoss(margin = 1.0)
# print(a.get_config())
# tf.assert_ a.call(np.array([1,0,1]), np.array([1,0,1])).numpy() == 0.0
# tf.assert_ a.call(np.array([0,0,0]), np.array([1,0,1])).numpy() >= 0.666
# tf.assert_ a.call(np.array([0,1,0]), np.array([1,0,1])).numpy() >= 1.0

In [15]:
def build_model2(model_type, embedding_dim, num_rnn_node, num_dense_node, num_layer, activation_fn, learning_rate, optimizer, margin):
    input_x = tf.keras.layers.Input(MAXLEN)
    input_1 = tf.keras.layers.Input(MAXLEN)
    input_2 = tf.keras.layers.Input(MAXLEN)
    embedding = tf.keras.layers.Embedding(input_dim=28, output_dim=embedding_dim, mask_zero=True)
    x = embedding(input_x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    if model_type == "lstm":
        x = tf.keras.layers.LSTM(num_rnn_node)(x)
    elif model_type=="gru":
        x = tf.keras.layers.GRU(num_rnn_node)(x)
 
    num = num_dense_node
    for _ in range(num_layer):
        x = tf.keras.layers.Dense(num, activation=activation_fn)(x)
        num /= 2
        
    embedding_network = tf.keras.Model(input_x, x)

    tower_1 = embedding_network(input_1)
    tower_2 = embedding_network(input_2)

    merge_layer = tf.keras.layers.Lambda(cosine_similarity)([tower_1, tower_2])
    normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
    output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(normal_layer)
    contr = tf.keras.Model(inputs=[input_1, input_2], outputs=output_layer)
    
    if optimizer == "Adam":
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer =="RMSprop":                
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer =="SGD":                
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)

    contr.compile(loss=ContrastiveLoss(margin = margin), optimizer=opt, metrics=["accuracy"])
    
    return contr

In [16]:
def plt_metric(history, metric, title, has_valid=True):
    """Plots the given 'metric' from 'history'.

    Arguments:
        history: history attribute of History object returned from Model.fit.
        metric: Metric to plot, a string value present as key in 'history'.
        title: A string to be used as title of plot.
        has_valid: Boolean, true if valid data was passed to Model.fit else false.

    Returns:
        None.
    """
    plt.plot(history[metric])
    if has_valid:
        plt.plot(history["val_" + metric])
        plt.legend(["train", "validation"], loc="upper left")
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel("epoch")
    plt.show()

# Build model, load weights and evaluate on test data

In [21]:
model = build_model2(
    model_type = "lstm", 
    embedding_dim = 10, 
    num_rnn_node = MAXLEN//2, 
    num_dense_node = 256, 
    num_layer = 1, 
    activation_fn = "tanh",
    learning_rate = 0.001,
    optimizer= "Adam",
    margin = 1.0
)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 400)]        0           []                               
                                                                                                  
 input_9 (InputLayer)           [(None, 400)]        0           []                               
                                                                                                  
 model_2 (Functional)           (None, 256)          220576      ['input_8[0][0]',                
                                                                  'input_9[0][0]']                
                                                                                                  
 lambda_1 (Lambda)              (None, 1)            0           ['model_2[0][0]',          

In [22]:
EPOCHS = 30
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='auto', patience=7)

In [23]:
# Model weights are saved at the end of every epoch, if it's the best seen so far.
history = model.fit(
    x=[np.stack(x1TrainRnnS), np.stack(x2TrainRnnS)],
    y=np.stack(yTrainRnnS),
    epochs=EPOCHS,
    batch_size=16,
    validation_data=([np.stack(x1ValRnnS), np.stack(x2ValRnnS)], np.stack(yValRnnS)),
    callbacks=[model_checkpoint_callback, early_stopping],
    use_multiprocessing=True
)

Epoch 1/30
1521/1521 [==============================] - 404s 263ms/step - loss: 0.1424 - accuracy: 0.8522 - val_loss: 0.1155 - val_accuracy: 0.8639
Epoch 2/30
1521/1521 [==============================] - 412s 271ms/step - loss: 0.1112 - accuracy: 0.8654 - val_loss: 0.1056 - val_accuracy: 0.8718
Epoch 3/30
1521/1521 [==============================] - 413s 271ms/step - loss: 0.1041 - accuracy: 0.8699 - val_loss: 0.1031 - val_accuracy: 0.8690
Epoch 4/30
1521/1521 [==============================] - 395s 260ms/step - loss: 0.1002 - accuracy: 0.8733 - val_loss: 0.1022 - val_accuracy: 0.8720
Epoch 5/30
1521/1521 [==============================] - 388s 255ms/step - loss: 0.0972 - accuracy: 0.8790 - val_loss: 0.1030 - val_accuracy: 0.8738
Epoch 6/30
1521/1521 [==============================] - 389s 256ms/step - loss: 0.0924 - accuracy: 0.8890 - val_loss: 0.1045 - val_accuracy: 0.8746
Epoch 7/30
1521/1521 [==============================] - 391s 257ms/step - loss: 0.0877 - accuracy: 0.8954 - val_

In [ ]:
# Plot the accuracy
plt_metric(history=history.history, metric="accuracy", title="Model accuracy")

In [ ]:
# Plot the constrastive loss
plt_metric(history=history.history, metric="loss", title="Constrastive Loss")

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
# model.save("../ModelCheckpointSaves/best_model.h5")
# model.load_weights("../exp3-hyperparameter-tuning/alstm-22-0.01.hdf5")